In [1]:
print()

In [155]:
# LOAD PACKAGES
# -------------
# import Pkg; Pkg.add("CSV"); Pkg.add("GLPK"); Pkg.add("JuMP")
using JuMP  # Need to say it whenever we use JuMP
using GLPK  # Loading the GLPK module for using its solver
using CSV
using Cbc

┌ Info: Precompiling Cbc [9961bab8-2fa3-5c5a-9d89-47fab24efd76]
└ @ Base loading.jl:1273


In [180]:
# MODEL CONSTRUCTION
# ------------------
capstone = Model(with_optimizer(Cbc.Optimizer, tm_lim=20*1000)) 
# We also set the time limit, in milliseconds

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: COIN Branch-and-Cut (Cbc)

In [181]:
# INPUT DATA
# ----------

# Data on student preferences
unit_costs = CSV.read("../preprocessing/unit_costs.csv",header=false)
print(size(unit_costs))

# Data on Pillars
capacities = CSV.read("../preprocessing/capacities.csv",header=false)
print(size(capacities))

# Data on Pillars
supplies = CSV.read("../preprocessing/supplies.csv",header=false)
print(size(supplies))

length = size(unit_costs)[1]
iterlen = 1:length

(6, 6)(6, 6)(6, 1)

1:6

In [182]:
# VARIABLES
# ---------

# the flow
@variable(capstone, flow[i=iterlen,j=iterlen] >= 0, Int)  

# binary variable taking value 1 if there is a flow
@variable(capstone, flowing[i=iterlen,j=iterlen], Bin)

print()

In [183]:
# CONSTRAINTS
# -----------

# flow can be nonzero only if flowing is 1
@constraint(capstone, smaller_1[i=iterlen, j=iterlen], 
    flow[i,j] <= flowing[i,j]*999)

# flowing can be nonzero only if capacity is nonzero
@constraint(capstone, smaller_2[i=iterlen, j=iterlen],
    flowing[i,j] <= capacities[i,j])

# Every node can only have one output
@constraint(capstone, output[i=1:(length-1)],
    sum(flowing[i,j] for j = iterlen) == 1)

# Every node needs to exhaust its supply
@constraint(capstone, supplied[i=iterlen],
    sum(flow[i,j] for j = iterlen) - sum(flow[j,i] for j = iterlen) == supplies[i,1])

print()

In [184]:
# OBJECTIVE
# ---------

# Maximize the total utility
@objective(capstone, Min, 
    sum(unit_costs[i,j]*flow[i,j] for i = iterlen, j = iterlen))

print(capstone) # Shows the model constructed in a human-readable form
print()

Min 14 flow[1,1] + 12 flow[1,2] + 7 flow[1,3] + 6 flow[1,4] + 9 flow[1,5] + 14 flow[1,6] + 7 flow[2,1] + 16 flow[2,2] + 12 flow[2,3] + 19 flow[2,4] + 11 flow[2,5] + flow[2,6] + 12 flow[3,1] + 19 flow[3,2] + 6 flow[3,3] + 18 flow[3,4] + 19 flow[3,5] + 14 flow[3,6] + 5 flow[4,1] + 19 flow[4,2] + 11 flow[4,3] + flow[4,4] + 11 flow[4,5] + 14 flow[4,6] + 3 flow[5,1] + 12 flow[5,2] + 5 flow[5,3] + 15 flow[5,4] + 12 flow[5,5] + 9 flow[5,6] + 12 flow[6,1] + 5 flow[6,2] + 6 flow[6,3] + 16 flow[6,4] + 15 flow[6,5] + 5 flow[6,6]
Subject to
 flowing[1,1] + flowing[1,2] + flowing[1,3] + flowing[1,4] + flowing[1,5] + flowing[1,6] = 1.0
 flowing[2,1] + flowing[2,2] + flowing[2,3] + flowing[2,4] + flowing[2,5] + flowing[2,6] = 1.0
 flowing[3,1] + flowing[3,2] + flowing[3,3] + flowing[3,4] + flowing[3,5] + flowing[3,6] = 1.0
 flowing[4,1] + flowing[4,2] + flowing[4,3] + flowing[4,4] + flowing[4,5] + flowing[4,6] = 1.0
 flowing[5,1] + flowing[5,2] + flowing[5,3] + flowing[5,4] + flowing[5,5] + flowing[5

In [185]:
# SOLVE IT
# --------

status = optimize!(capstone)
print()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Oct  7 2019 

command line - Cbc_C_Interface -tm_lim 20000 -solve -quit (default strategy 1)
No match for tm_lim - ? for list of commands
No match for 20000 - ? for list of commands
Continuous objective value is 158 - 0.00 seconds
Cgl0002I 23 variables fixed
Cgl0003I 0 fixed, 4 tightened bounds, 4 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 4 tightened bounds, 4 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 6 tightened bounds, 6 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 6 tightened bounds, 6 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 4 tightened bounds, 4 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 6 tightened bounds, 5 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 5 tightened bounds, 5 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 6 tightened bounds, 5 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 5 tightened bounds, 5 strengthened rows, 0 substitutions
Cgl00

In [186]:
# DISPLAY THE RESULTS
# -------------------

println("Objective value: ", JuMP.objective_value(capstone))
println("flow = ", JuMP.value.(flow))
println("flowing = ", JuMP.value.(flowing))

Objective value: 158.0
flow = 2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, 1:6
    Dimension 2, 1:6
And data, a 6×6 Array{Float64,2}:
 0.0  0.9999999999999999  0.0  0.0  0.0  0.0               
 0.0  0.0                 0.0  0.0  0.0  2.9999999999999996
 0.0  0.0                 0.0  0.0  0.0  2.9999999999999996
 0.0  0.0                 0.0  0.0  0.0  3.9999999999999996
 0.0  0.0                 0.0  0.0  0.0  5.0               
 0.0  0.0                 0.0  0.0  0.0  0.0               
flowing = 2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, 1:6
    Dimension 2, 1:6
And data, a 6×6 Array{Float64,2}:
 0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0
 0.0  0.0  0.0  0.0  0.0  1.0
 0.0  0.0  0.0  0.0  0.0  1.0
 0.0  0.0  0.0  0.0  0.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0
